# Gender Expression

### About Dataset
##### From: [Dataset](http://www.zhuolin.umiacs.io/projectlcksvd.html)
Bộ cơ sở dữ liệu này bao gồm 2600 ảnh, chia làm 100 lớp (mỗi lớp ứng với một người), tức mỗi người có 26 ảnh
Mỗi bức ảnh trong AR Face thu gọn được đặt tên dưới dạng G-xxx-yy.bmp
- G nhận một trong hai giá trị M (man) hoặc W (woman)
- xxx là id của người, nhận gía trị từ 001 đến 126
- yy là điều kiện chụp, nhận giá trị từ 01 đến 26<br>

Desciption:
(1) featureMat:(540x2600)
A matrix of random features. Each column is random face feature vector

(2) filenameMat 1x100 cells (cell 1x26)
Image file names. Each cell correspond to the features from the same class in 'featureMat'. 

(3) labelMat:
This is a label matrix, each column corresponds to one random face feature, where the non-zero position of each column 
indicates the class of the random face feature.



In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import linear_model 
from sklearn.metrics import accuracy_score
from scipy import misc      
from scipy.io import loadmat
from sklearn import preprocessing
np.random.seed(1)


### Load data

In [3]:
path = 'dataset/faces'
faces_data = loadmat(path)
featureMat = faces_data['featureMat']
filenameMat = faces_data['filenameMat'][0]
labelMat = faces_data['labelMat']

d = 540 #dimension 
N = 2600 # the number of feature vector
randID = np.random.permutation(np.arange(0, N))
train_ids = randID[0:int(N/2)]
train_size = train_ids.shape[0]
test_ids = randID[int(N/2) + 1:]
test_size = test_ids.shape[0]
view_ids = np.arange(0, 26 )

gender = np.char.split(filenameMat[9][0][0][0], sep='-')
gender = np.char.split(filenameMat[6][0][0][0], sep='-')

gender.tolist()[0]

'M'

## Handle Data 

In [4]:
def build_data_matrix(vector_ids, d, size, featureMat, filenameMat, labelMat ):
    X = np.empty((d,1))
    y = np.empty((1, 1))
    for j in vector_ids:
        X = np.concatenate((X, featureMat[:,j].reshape(d,1)), axis = 1)
        person = np.argwhere(labelMat[:,j] == 1)
        gender = np.char.split(filenameMat[person[0][0]][0][0][0], sep='-').tolist()[0]
        if (gender == 'M'):
            y = np.concatenate((y, [[1]]), axis = 0)
        else:
            y = np.concatenate((y, [[0]]), axis = 0)
    X = X[:, 1:]
    y = y[1:, :].flatten()
    return (X, y)


X_train, y_train = build_data_matrix(train_ids, d, train_size, featureMat, filenameMat, labelMat)  
X_test, y_test = build_data_matrix(test_ids, d, test_size, featureMat, filenameMat, labelMat)  


## Logistic Regression - Step by step

In [43]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def logistic_sigmoid_regression(X_bar, y, w_init, eta, tol = 1e-8, max_count = 1000000):
    w = [w_init]    
    it = 0
    N = X_bar.shape[1]
    d = X_bar.shape[0]
    count = 0
    check_w_after = 2600
    while count < max_count:
        # mix data for SGD 
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X_bar[:, i].reshape(d, 1)
            yi = y[i]
            zi = sigmoid(np.dot(w[-1].T, xi))
            w_new = w[-1] + eta*(yi - zi)*xi
            count += 1
            # stopping criteria
            if count%check_w_after == 0:                
                if np.linalg.norm(w_new - w[-check_w_after]) < tol:
                    return w
            w.append(w_new)
    return w


eta = .05 
X_train_bar= np.concatenate((np.ones((1, X_train.shape[1])), X_train), axis = 0)
X_test_bar= np.concatenate((np.ones((1, X_test.shape[1])), X_test), axis = 0)
d = X_train_bar.shape[0]
w_init = np.random.randn(d, 1)
w = logistic_sigmoid_regression(X_train_bar, y_train, w_init, eta)

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_11536\818875205.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-s))


### Review

In [44]:
w0 = w[-1]

def review(w, X_bar, y, size):
    y_pre = np.ones(size)
    for i in range(size):
        xi = X_bar[:, i].reshape(d, 1)
        zi = sigmoid(np.dot(w.T, xi))
        if (zi < 0.5):
            y_pre[i] = 0
    print ("Accuracy: %.2f %%" %(100*accuracy_score(y, y_pre)))

review(w0, X_test_bar, y_test, test_size)

Accuracy: 96.69 %


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_11536\818875205.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-s))


## Using  linear_model.LogisticRegression
Chú ý: Hàm sklearn.linear_model.LogisticRegression nhận dữ liệu ở dạng vector hàng.

In [22]:
logreg = linear_model.LogisticRegression(C=1e5) # just a big number 
logreg.fit(X_train.T, y_train) #Mo

y_pred = logreg.predict(X_test.T)
print ("Accuracy: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy: 95.46 %
